# Evaluación del etiquetado: columna `sentimiento_5`

El objetivo de este cuaderno es evaluar la calidad del etiquetado en la columna `sentimiento_5`. Se busca verificar, mediante una prueba de hipótesis, si el etiquetado original es consistente con un etiquetado realizado offline sobre una muestra representativa.

Flujo de trabajo:
- Carga de datos
- Exportación de muestra para etiquetar offline
- Normalización de etiquetas (original y offline)
- Reintegración de la muestra etiquetada
- Métricas de concordancia
- Prueba de hipótesis (una cola) sobre la exactitud

## Guía para etiquetado offline

- Generación del CSV para etiquetar: sección “Exportar muestra para etiquetado offline”. Se crea `muestra_2080_para_etiquetar.csv` con `row_id`, `Review` y `sentimiento_5_offline` vacío.
- Carga del CSV ya etiquetado: sección “Reinsertar CSV con la muestra etiquetada offline”. Se lee el archivo (por defecto `muestra_2080_etiquetado_offline.csv`), se validan columnas y se reintegran las etiquetas al DataFrame.

In [1]:
# Importación de librerías
import pandas as pd  # manejo de datos tabulares
import numpy as np   # utilidades numéricas
from scipy import stats  # funciones estadísticas (normal, percentiles, etc.)
from sklearn.metrics import confusion_matrix, classification_report  # métricas de clasificación

In [15]:
indicadores = pd.read_csv('indicadores_turismo_completo.csv')
head_df = indicadores.head()

## Carga de datos: columnas esperadas y validaciones

En este paso se lee el archivo CSV de origen y se verifican columnas y forma.
- Archivo: `FILE_PATH` (por defecto `Rest-Mex_con_analisis.csv`).
- Columnas esperadas: `Title`, `Review`, `Polarity`, `Town`, `Region`, `Type`, `sentimiento_5`.
- Se imprime forma y nombres de columnas para validación rápida.

In [2]:
# Carga de datos
    # La ruta al archivo puede ajustarse según necesidad
FILE_PATH = 'Rest-Mex_con_analisis.csv'

try:
    # Lectura del CSV con las columnas de interés
    df = pd.read_csv(FILE_PATH)
    print(f"Datos cargados desde: {FILE_PATH}")
    print(f"Filas: {len(df):,} | Columnas: {len(df.columns)}")
    print("Columnas:", list(df.columns))
except FileNotFoundError:
    # En caso de no existir el archivo, el flujo se detiene con un mensaje claro
    raise FileNotFoundError(
        f"No se encontró el archivo '{FILE_PATH}'. Colóquelo en este directorio o actualice FILE_PATH.")

# Vista rápida para validar la lectura del DataFrame
df.head()

Datos cargados desde: Rest-Mex_con_analisis.csv
Filas: 208,051 | Columnas: 7
Columnas: ['Title', 'Review', 'Polarity', 'Town', 'Region', 'Type', 'sentimiento_5']


,Title,Review,Polarity,Town,Region,Type,sentimiento_5
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant,0.977972
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive,0.941615
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel,0.916443
3,un buen panorama cerca de Cancún,"Estando en Cancún, fuimos al puerto y tomamos ...",4.0,Isla_Mujeres,QuintanaRoo,Attractive,0.656095
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel,0.976754


## Exportar muestra para etiquetado offline

En este paso se genera el CSV que será etiquetado fuera de línea.
- Tamaño de la muestra: 2080 comentarios al azar (1% del total de datos)
- Columnas: `row_id` (índice original), `Review` y `sentimiento_5_offline` (vacía).
- Se ignoran columnas de puntaje.
- Archivo generado: `muestra_2080_para_etiquetar.csv`.

In [3]:
# Crear y exportar muestra de 2080 comentarios para etiquetado offline (sin puntajes)
TEXT_COL = 'Review'  # especifico que usaré la columna de texto 'Review'

# Valido que la columna de texto exista
a = list(df.columns)
if TEXT_COL not in a:
    raise KeyError(f"No se encontró la columna de texto '{TEXT_COL}'. Columnas disponibles: {a}")

# Tomo una muestra de 2080 y conservo el índice original en 'row_id'
muestra = df.sample(n=2080, random_state=42).reset_index().rename(columns={'index': 'row_id'})

# Agrego la columna vacía que será etiquetada offline
muestra['sentimiento_5_offline'] = ''

# Exporto solo las columnas necesarias
export_cols = ['row_id', TEXT_COL, 'sentimiento_5_offline']
export_path = 'muestra_2080_para_etiquetar.csv'
muestra[export_cols].to_csv(export_path, index=False, encoding='utf-8')

print(f"Muestra exportada a: {export_path}")
print(f"Columnas exportadas: {export_cols}")
print("Etiquete 'sentimiento_5_offline' con: Muy Negativo, Negativo, Neutral, Positivo, Muy Positivo.")
# Muestro las primeras filas para ver que la estructura es correcta
# tamaño
print(f"Tamaño de la muestra: {len(muestra):,} filas")
muestra.head()

Muestra exportada a: muestra_2080_para_etiquetar.csv
Columnas exportadas: ['row_id', 'Review', 'sentimiento_5_offline']
Etiquete 'sentimiento_5_offline' con: Muy Negativo, Negativo, Neutral, Positivo, Muy Positivo.
Tamaño de la muestra: 2,080 filas


,row_id,Title,Review,Polarity,Town,Region,Type,sentimiento_5,sentimiento_5_offline
0,161289,MEXICANO,"Es un lugar bellísimo, para llegar es necesari...",4.0,Patzcuaro,Michoacan,Attractive,0.951432,
1,10063,Una vista sensacional,"El lugar es muy agradable, con una vista realm...",3.0,Tepoztlan,Morelos,Restaurant,0.963154,
2,80240,Bastante recomendable para pasar la tarde,Es una plaza en forma de herradura con muchos ...,4.0,Ixtapan_de_la_Sal,Estado_de_Mexico,Attractive,0.769188,
3,201614,Dreams Tulum - algo para destacar,Excelente atención de Miguel Manzanero en el S...,4.0,Tulum,QuintanaRoo,Hotel,0.973364,
4,176112,Excelente servicio,El desayuno esta súper rico y el servicio fue ...,5.0,Metepec,Estado_de_Mexico,Restaurant,0.974753,


## Preparación y validación de etiquetas (original)

Se normaliza la columna `sentimiento_5` para asegurar el uso de 5 categorías estándar: `Muy Negativo`, `Negativo`, `Neutral`, `Positivo`, `Muy Positivo`.
- Se admiten valores numéricos (-2..2) y cadenas con o sin acentos.
- Se reporta la distribución y se filtran filas válidas.
- Se crea `sentimiento_5_norm` y `df_eval` para evaluar.

In [4]:
# Normalización de `sentimiento_5`
import unicodedata  # para eliminar acentos

REQUIRED_COLUMN = 'sentimiento_5'
TARGET_LABELS = ['Muy Negativo', 'Negativo', 'Neutral', 'Positivo', 'Muy Positivo']

# Verifico que exista la columna requerida
if REQUIRED_COLUMN not in df.columns:
    raise KeyError(
        f"No existe la columna '{REQUIRED_COLUMN}' en el DataFrame. Añádala al CSV para continuar.")

# Función para quitar acentos
def strip_accents(text):
    if not isinstance(text, str):
        return text
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

# Función principal de normalización
def normalize_sent_5(x):
    # Manejo de faltantes
    if pd.isna(x):
        return np.nan
    # Si viene numérico, mapeo -2..2 a las 5 categorías
    if isinstance(x, (int, float, np.integer, np.floating)):
        try:
            xi = int(np.clip(int(round(x)), -2, 2))
        except Exception:
            return np.nan
        num_map = {
            -2: 'Muy Negativo',
            -1: 'Negativo',
             0: 'Neutral',
             1: 'Positivo',
             2: 'Muy Positivo'
        }
        return num_map.get(xi, np.nan)

    # Si viene cadena, elimino acentos y normalizo espacios/ separadores
    s = strip_accents(str(x)).strip().lower()
    synonyms = {
        'muy negativo': 'Muy Negativo',
        'mn': 'Muy Negativo',
        'negativo': 'Negativo',
        'neg': 'Negativo',
        'neutral': 'Neutral',
        'neu': 'Neutral',
        'positivo': 'Positivo',
        'pos': 'Positivo',
        'muy positivo': 'Muy Positivo',
        'mp': 'Muy Positivo'
    }
    s = s.replace('_', ' ').replace('-', ' ')
    s = ' '.join(s.split())
    return synonyms.get(s, np.nan)

# Aplico la normalización sobre la columna original
col_norm = REQUIRED_COLUMN + '_norm'
df[col_norm] = df[REQUIRED_COLUMN].apply(normalize_sent_5)

# Reporto la distribución para revisar calidad
print('Distribución de etiquetas normalizadas (original):')
print(df[col_norm].value_counts(dropna=False))

# Genero el subconjunto válido para evaluación
df_eval = df.loc[df[col_norm].isin(TARGET_LABELS)].copy()
print(f"\nFilas válidas para evaluación: {len(df_eval):,} de {len(df):,}")

if len(df_eval) == 0:
    raise ValueError('No hay filas con etiquetas válidas en `sentimiento_5`. Revise el mapeo o sus datos.')

# Vista rápida del DF de evaluación
df_eval.head()

Distribución de etiquetas normalizadas (original):
sentimiento_5_norm
Positivo    175326
Negativo     17780
Neutral      14945
Name: count, dtype: int64

Filas válidas para evaluación: 208,051 de 208,051


,Title,Review,Polarity,Town,Region,Type,sentimiento_5,sentimiento_5_norm
0,Mi Lugar Favorito!!!!,Excelente lugar para comer y pasar una buena n...,5.0,Sayulita,Nayarit,Restaurant,0.977972,Positivo
1,lugares interesantes para visitar,"andar mucho, así que un poco difícil para pers...",4.0,Tulum,QuintanaRoo,Attractive,0.941615,Positivo
2,No es el mismo Dreams,"Es nuestra cuarta visita a Dreams Tulum, elegi...",3.0,Tulum,QuintanaRoo,Hotel,0.916443,Positivo
3,un buen panorama cerca de Cancún,"Estando en Cancún, fuimos al puerto y tomamos ...",4.0,Isla_Mujeres,QuintanaRoo,Attractive,0.656095,Positivo
4,El mejor,Es un lugar antiguo y por eso me encanto tiene...,5.0,Patzcuaro,Michoacan,Hotel,0.976754,Positivo


## Reinsertar CSV con la muestra etiquetada offline

Se carga el archivo etiquetado fuera de línea y se une con el DataFrame original por `row_id`. Se normaliza la etiqueta offline y la evaluación se realiza sobre la muestra reintegrada.

In [5]:
df_off = pd.read_csv('muestra_2080_etiquetado_offline.csv')
# Conservar solo las columnas necesarias
df_off = df_off[['row_id', 'sentiment']]
df_off = df_off.rename(columns={'sentiment': 'sentimiento_5_offline'})

df_off.head()

,row_id,sentimiento_5_offline
0,161289,0.779023
1,10063,0.802450
2,80240,0.858585
3,201614,0.953817
4,176112,0.895559


In [6]:
# Uso la misma función de normalización para la etiqueta offline
if 'normalize_sent_5' not in globals():
    raise RuntimeError("Ejecute primero la celda de normalización de `sentimiento_5`.")

df_off['sentimiento_5_offline_norm'] = df_off['sentimiento_5_offline'].apply(normalize_sent_5)
print('Distribución etiquetas offline normalizadas:')
print(df_off['sentimiento_5_offline_norm'].value_counts(dropna=False))

# Uno con el DF original a partir del índice preservado en la exportación
df_with_id = df.reset_index().rename(columns={'index': 'row_id'})
df_sample_merged = df_with_id.merge(
    df_off[['row_id', 'sentimiento_5_offline', 'sentimiento_5_offline_norm']],
    on='row_id', how='inner'
)

# Aseguro que la columna normalizada original esté presente
if 'sentimiento_5_norm' not in df_sample_merged.columns:
    df_sample_merged['sentimiento_5_norm'] = df_sample_merged['sentimiento_5'].apply(normalize_sent_5)

# A partir de aquí, evalúo exclusivamente sobre la muestra etiquetada offline
df_eval = df_sample_merged.copy()

print(f"\n Reintegración completa. Filas muestras: {len(df_sample_merged):,}")
print('Columnas clave:', ['sentimiento_5_norm', 'sentimiento_5_offline_norm'])
# Vista preliminar de la muestra reintegrada
df_sample_merged.head()

Distribución etiquetas offline normalizadas:
sentimiento_5_offline_norm
Positivo    1540
Neutral      307
Negativo     233
Name: count, dtype: int64

 Reintegración completa. Filas muestras: 2,080
Columnas clave: ['sentimiento_5_norm', 'sentimiento_5_offline_norm']


,row_id,Title,Review,Polarity,Town,Region,Type,sentimiento_5,sentimiento_5_norm,sentimiento_5_offline,sentimiento_5_offline_norm
0,105,Casa lejos de casa!,"Visitamos La Zebra en 2015 diciembre, justo an...",5.0,Tulum,QuintanaRoo,Hotel,0.905953,Positivo,0.838202,Positivo
1,238,Hermoso,"Al entrar, hay personas que te hacen pensar qu...",5.0,Tulum,QuintanaRoo,Attractive,0.663607,Positivo,0.609490,Positivo
2,272,Estupendo,"Increible el comer ahi, que gastronomia y que ...",5.0,Valladolid,Yucatan,Restaurant,0.977989,Positivo,0.922786,Positivo
3,285,Impresionantemente hermoso,La primera vez que Izamul (4ª vez a Mérida); n...,5.0,Izamal,Yucatan,Attractive,0.940970,Positivo,0.612713,Positivo
4,483,increíble,Si en México su bien vale la pena tomar el tie...,5.0,Tulum,QuintanaRoo,Attractive,0.972450,Positivo,0.796035,Positivo


## Métricas de concordancia

Se calcula la exactitud comparando la etiqueta original normalizada (`sentimiento_5_norm`) contra la etiqueta offline normalizada (`sentimiento_5_offline_norm`). Se reporta además la matriz de confusión y el reporte de clasificación.

In [7]:
# Cálculo de exactitud, matriz de confusión y reporte (original vs offline)
LABELS_ORDER = ['Muy Negativo', 'Negativo', 'Neutral', 'Positivo', 'Muy Positivo']

# Verifico que estén las columnas normalizadas necesarias
needed_cols = ['sentimiento_5_norm', 'sentimiento_5_offline_norm']
for c in needed_cols:
    if c not in df_eval.columns:
        raise KeyError(f"Falta la columna {c} en df_eval. Revise la reinserción del CSV offline.")

# Me quedo con las filas válidas (ambas etiquetas dentro del catálogo)
mask_valid_metrics = df_eval['sentimiento_5_norm'].isin(LABELS_ORDER) & df_eval['sentimiento_5_offline_norm'].isin(LABELS_ORDER)
df_metrics = df_eval.loc[mask_valid_metrics].copy()

if len(df_metrics) == 0:
    raise ValueError('No hay filas válidas para calcular métricas (revisar normalización).')

# Defino ground truth como la etiqueta offline y evalúo la original contra esa referencia
y_true = df_metrics['sentimiento_5_offline_norm']  # referencia (offline)
y_pred = df_metrics['sentimiento_5_norm']          # etiquetado original a evaluar

# Exactitud como proporción de coincidencias
accuracy = (y_true == y_pred).mean()
print(f"Total observaciones válidas: {len(df_metrics):,}")
print(f"Exactitud (accuracy): {accuracy:.4f}")

# Matriz de confusión para entender los desacuerdos por clase
mat = confusion_matrix(y_true, y_pred, labels=LABELS_ORDER)
print("\nMatriz de Confusión (Filas: offline, Columnas: original):")
print(pd.DataFrame(mat, index=LABELS_ORDER, columns=LABELS_ORDER))

# Reporte de clasificación con precisión, recall y F1 por clase
print("\nReporte de clasificación (original vs offline):")
print(classification_report(y_true, y_pred, labels=LABELS_ORDER, target_names=LABELS_ORDER, zero_division=0))

Total observaciones válidas: 2,080
Exactitud (accuracy): 0.8635

Matriz de Confusión (Filas: offline, Columnas: original):
              Muy Negativo  Negativo  Neutral  Positivo  Muy Positivo
Muy Negativo             0         0        0         0             0
Negativo                 0       162        9        62             0
Neutral                  0        35      120       152             0
Positivo                 0         7       19      1514             0
Muy Positivo             0         0        0         0             0

Reporte de clasificación (original vs offline):
              precision    recall  f1-score   support

Muy Negativo       0.00      0.00      0.00         0
    Negativo       0.79      0.70      0.74       233
     Neutral       0.81      0.39      0.53       307
    Positivo       0.88      0.98      0.93      1540
Muy Positivo       0.00      0.00      0.00         0

    accuracy                           0.86      2080
   macro avg       0.50     

## Diagnóstico de exactitud y discrepancias

Cuando la exactitud no es la esperada, se analiza la distribución de etiquetas y se muestran ejemplos de discrepancias para identificar sesgos o errores de etiquetado.

In [8]:
# Exploración detallada de discrepancias (offline vs original)
LABELS_ORDER = ['Muy Negativo', 'Negativo', 'Neutral', 'Positivo', 'Muy Positivo']

# Verifico que tenga ambas columnas normalizadas
if 'sentimiento_5_offline_norm' not in df_eval.columns or 'sentimiento_5_norm' not in df_eval.columns:
    raise RuntimeError("Faltan columnas normalizadas. Ejecute reinserción y normalización.")

# Creo un subconjunto con nombres más claros para imprimir
subset = df_eval[['sentimiento_5_offline_norm', 'sentimiento_5_norm']].copy().rename(columns={
    'sentimiento_5_offline_norm': 'offline',
    'sentimiento_5_norm': 'original'
})

# Conteos básicos para entender distribución
print('Conteo etiquetas offline:')
print(subset['offline'].value_counts(dropna=False))
print('\nConteo etiquetas original:')
print(subset['original'].value_counts(dropna=False))

# Pairs válidos dentro del catálogo
mask_valid = subset['offline'].isin(LABELS_ORDER) & subset['original'].isin(LABELS_ORDER)
valid_pairs = subset[mask_valid]

if len(valid_pairs) == 0:
    print('\n⚠ No hay pares válidos (posibles NaNs o etiquetas fuera del catálogo).')
else:
    # Crosstab para ver patrones de desacuerdo
    crosstab = pd.crosstab(valid_pairs['offline'], valid_pairs['original'], dropna=False)
    print('\nCrosstab offline vs original:')
    print(crosstab)

    # Ejemplos de discrepancias
    mismatches = valid_pairs[valid_pairs['offline'] != valid_pairs['original']]
    print(f"\nTotal mismatches: {len(mismatches):,}")
    if len(mismatches) > 0:
        print('\nEjemplos de discrepancias (hasta 10):')
        print(mismatches.head(10))

# Reviso valores únicos en ambos lados
unique_offline = set(df_eval['sentimiento_5_offline_norm'].dropna().unique())
unique_original = set(df_eval['sentimiento_5_norm'].dropna().unique())
print('\nValores únicos (offline):', unique_offline)
print('Valores únicos (original):', unique_original)

# Mensaje de ayuda según conjuntos
if unique_offline == unique_original:
    print('\n✔ Conjuntos de categorías coinciden. Las diferencias reflejan desacuerdo de etiquetado.')
else:
    print('\n⚠ Conjuntos de categorías NO coinciden. Revisar mapeo offline vs catálogo esperado.')

# Sugerencias si la exactitud salió 0
try:
    if 'accuracy' in globals():
        if accuracy == 0 and len(valid_pairs) > 0:
            print('\n🔎 Accuracy = 0 con pares válidos. Posibles causas:')
            print('- Etiquetado offline sistemáticamente diferente al criterio original.')
            print('- Diferencias semánticas en interpretación de categorías.')
            print('- Errores al etiquetar offline (revisar ejemplos de discrepancias).')
except Exception:
    pass

Conteo etiquetas offline:
offline
Positivo    1540
Neutral      307
Negativo     233
Name: count, dtype: int64

Conteo etiquetas original:
original
Positivo    1728
Negativo     204
Neutral      148
Name: count, dtype: int64

Crosstab offline vs original:
original  Negativo  Neutral  Positivo
offline                              
Negativo       162        9        62
Neutral         35      120       152
Positivo         7       19      1514

Total mismatches: 284

Ejemplos de discrepancias (hasta 10):
     offline  original
6   Negativo  Positivo
12  Negativo  Positivo
13   Neutral  Positivo
16   Neutral  Positivo
41   Neutral  Positivo
46   Neutral  Negativo
52   Neutral  Positivo
61   Neutral  Positivo
67  Negativo  Positivo
68   Neutral  Positivo

Valores únicos (offline): {'Neutral', 'Positivo', 'Negativo'}
Valores únicos (original): {'Neutral', 'Positivo', 'Negativo'}

✔ Conjuntos de categorías coinciden. Las diferencias reflejan desacuerdo de etiquetado.


## Prueba de hipótesis sobre la exactitud (original vs offline)

Se formula una prueba z de proporción (una cola) para evaluar si la exactitud del etiquetado original supera un umbral mínimo `p0`.
- H0: p ≤ p0
- Ha: p > p0
- p̂: proporción de coincidencias entre `sentimiento_5_norm` (original) y `sentimiento_5_offline_norm` (offline).

In [9]:
# Z-test una cola para proporción (exactitud original vs offline)
p0 = 0.40  # Umbral mínimo que considero aceptable (ajustable)
alpha = 0.05  # Nivel de significación

n = len(df_metrics)  # tamaño muestral para la prueba
p_hat = accuracy      # proporción observada de aciertos

# Calculo el error estándar bajo H0
std_err = np.sqrt(p0 * (1 - p0) / n)
if std_err == 0:
    raise ZeroDivisionError('Error estándar 0 (revisar n y p0).')

# Estadístico z y p-valor de una cola
z_stat = (p_hat - p0) / std_err
p_value = 1 - stats.norm.cdf(z_stat)

print(f"H0: p ≤ {p0:.2f} | Ha: p > {p0:.2f}")
print(f"n = {n}")
print(f"p̂ (exactitud observada) = {p_hat:.4f}")
print(f"Z = {z_stat:.4f}")
print(f"p-valor (una cola) = {p_value:.4f}")

# Decisión de la prueba
a if p_value < alpha else None
if p_value < alpha:
    decision = 'RECHAZAR H0'
    rationale = f"Concluyo que la exactitud > {p0:.2f} al nivel {alpha}."
else:
    decision = 'NO RECHAZAR H0'
    rationale = f"No hay evidencia suficiente para concluir que la exactitud > {p0:.2f} al nivel {alpha}."

print(f"Decisión: {decision}")
print(f"Conclusión: {rationale}")

# Intervalo de confianza 95% para p̂ (aprox. normal)
z_crit = stats.norm.ppf(1 - 0.025)
se_hat = np.sqrt(p_hat * (1 - p_hat) / n)
ci_low = max(0.0, p_hat - z_crit * se_hat)
ci_high = min(1.0, p_hat + z_crit * se_hat)
print(f"IC 95% para la exactitud: [{ci_low:.4f}, {ci_high:.4f}]")

H0: p ≤ 0.40 | Ha: p > 0.40
n = 2080
p̂ (exactitud observada) = 0.8635
Z = 43.1459
p-valor (una cola) = 0.0000
Decisión: RECHAZAR H0
Conclusión: Concluyo que la exactitud > 0.40 al nivel 0.05.
IC 95% para la exactitud: [0.8487, 0.8782]
